# Labs workforce forecasting (very basic) simulation model

TBI:               
all bugs fixed, relevant commenting added throughout for clarity

the code should than run a very basic simulation model with only 1 type (band 3) of employees, dealing with uniform tasks at the given rate    
it should allow for building on top of it, adding various complications structurally- and technically-wise            
 
the next step would be adding employees of band 2, who would perform the same tasks, but at a higher rate      

than band 1 would be added, who would perform a different job - verify processed samples (completed work) from bands 3 and 2.          
band 1 would also deal with occasionally emerging complex samples, that would occupy a significant portion of their time      

so there would have to be two queues or potential bottlenecks in this model:            
one - at the very entry to the lab, indicating insufficient overall performance of band 3 + band 2 workers                      
two - verification queue from band 3 + band 2 to band 1. potentially elevating (non-existant yet) issue of analysing complex samples in the context of intense verification work
____________________________________________________________________________________________________________
the runtime of the model would be 1 year = 365 days         
as a typical workday lasts for 8 hours, appropriate changes have to be made to the code

all parameters should be fine-tuned to resemble a somewhat realistic model, this could only be done after obtaining some results           
ideally, they should be calculated based on the real-world data so the model is very similar to real-world lab conditions and produced results are realistic and meaningful

warm-up period of the model should be implemented to assess the time when it reaches a stable state (all workers are loaded, queue times are stable etc)       
number of runs for the model should be calculated to produce reliable and accurte results, estimate confidence intervals     


<b>(draw idea schematically)</b>      

Three types of employees:

First line__________________

Band 3 (noobs):                   x5                  
Process samples              
Avg time - 1 / sample

Band 2 (pros):                    x2                        
Process samples                     
Avg time - 0,5 / sample

Second line_______________

Band 1 (alpha):                    x1           
Verify results               
Avg time - 0,05 / sample             
Process complex sample                
Avg time - 0,75 / sample

Chances of complex sample - 0,02         
Mean samples inter-arrival time - 0,1        


## Imports

In [1]:
import numpy as np
import pandas as pd
import itertools
import math
import matplotlib.pyplot as plt
import simpy
from joblib import Parallel, delayed
import warnings
from scipy.stats import t
from treat_sim.distributions import Exponential, Lognormal

## Utility functions

In [2]:
def trace(msg):
    '''
    Utility function for printing simulation
    set the TRACE constant to FALSE to 
    turn tracing off.
    
    Params:
    -------
    msg: str
        string to print to screen.
    '''
    if TRACE:
        print(msg)

## Model parameters
 

In [3]:
# These are the parameters for a base case model run.

# run length in days
RUN_LENGTH = 365

# resource counts
N_WORKERS = 5

# time between arrivals in hours (exponential)
MEAN_IAT = 2

# processing time (lognormal)
PROCESS_MEAN = 1
PROCESS_STD = 1.2

# default № of reps for multiple reps run
DEFAULT_N_REPS = 5

# default random number SET
DEFAULT_RNG_SET = None
N_STREAMS = 10

# Turn off tracing
TRACE = False

## Scenario class

In [4]:
class Scenario:

    def __init__(self, random_number_set=DEFAULT_RNG_SET):
        
        # Warm-up period
        self.warm_up = 0.0

        # Default values for inter-arrival and process times
        self.iat_mean = MEAN_IAT
        self.process_mean = PROCESS_MEAN
        self.process_std = PROCESS_STD

        # Sampling
        self.random_number_set = random_number_set
        self.init_sampling()

        # Number of emplyees
        self.n_workers = N_WORKERS

    def set_random_no_set(self, random_number_set):
        '''
        Set the random number set to be used by the simulation.

        Parameters:
        ----------
        random_number_set: int
            The random number set to be used by the simulation.
        '''
        self.random_number_set = random_number_set
        self.init_sampling()

    def init_sampling(self):
        '''
        Initialize the random number streams and create the distributions used by the simulation.
        '''

        # Create random number streams
        rng_streams = np.random.default_rng(self.random_number_set)

        # Initialize the random seeds for each stream
        self.seeds = rng_streams.integers(0, 999999999, size=N_STREAMS)

        # Create inter-arrival time distribution
        self.arrival_dist = Exponential(self.iat_mean, random_seed=self.seeds[0])

        # Create process time distributions 
        self.process_dist = Lognormal(self.process_mean, self.process_std, random_seed=self.seeds[1])

## Model building

In [5]:
class Sample:

    def __init__(self, identifier, env, args):

        # sample id and environment
        self.identifier = identifier
        self.env = env
        
        # processing parameters
        self.workers = args.workers
        self.process_dist = args.process_dist
    
    def get_process_dist_sample(self):
        '''
        This method returns a sample from the process distribution of the sample, based on the employee's band.
        '''
        self.process_time = self.process_dist.sample()
        return self.process_time
    
    def process(self):
        '''
        This method represents the sample processing. The sample arrives to the lab, waits for an available worker to take it into work, wait in the queue,
        and then undergo processing before being passed for evaluation.
        '''
        # record the time when sample entered the system
        arrival_time = self.env.now
     
        # take sample into work
        with self.workers.request() as req:
            yield req
            
            # calculate queue time and log it
            self.queue_time = self.env.now - arrival_time
            trace(f'Sample № {self.identifier} taken into work at {self.env.now:.3f};' 
                 + f' queue time was {self.queue_time:.3f}') 
            
            # wait for processing to finish
            yield self.env.timeout(self.get_process_dist_sample())

In [9]:
class LAB:

    def __init__(self, args):

        self.env = simpy.Environment()
        self.args = args 
        self.process_dist = args.process_dist
        self.init_model_resources()
        self.samples = []
        
        self.arrivals_count = 0
        
        
    def init_model_resources(self):

        self.workers = simpy.Resource(self.env, 
                                   capacity=self.args.n_workers)
        
    def run(self, results_collection_period = RUN_LENGTH,
            warm_up = 0):
        
        # setup the arrival processes
        self.env.process(self.arrivals_generator())
                
        # run
        self.env.run(until=results_collection_period+warm_up)
        
        
    def get_arrival_dist_sample(self):
        
        inter_arrival_time = self.args.arrival_dist.sample()
        return inter_arrival_time
                
    def arrivals_generator(self):
        self.args.init_sampling()
            
        while True: 

            iat = self.get_arrival_dist_sample()
            yield self.env.timeout(iat)
                
            if self.env.now > self.args.warm_up:    
                self.arrivals_count += 1

            trace(f'Sample № {self.arrivals_count}  arrives at {self.env.now:.3f}')
                
            new_sample = Sample(self.env, self.args, self)

            self.env.process(new_sample.process()) 

## Function for single run

In [10]:
def single_run(scenario, 
               rc_period = RUN_LENGTH, 
               warm_up = 0,
               random_no_set = DEFAULT_RNG_SET):
        
    # set random number set - this controls sampling for the run.
    if random_no_set is not None:
        scenario.set_random_no_set(random_no_set)
    
    scenario.warm_up = warm_up
    
    # create the model
    model = LAB(scenario)

    model.run(results_collection_period = rc_period, warm_up = warm_up)
    
    # run the model
    results_summary= model.run_summary_frame()
    
    return results_summary

# Scripts to run the model

In [11]:
# SINGLE RUN

# base case scenario with default parameters
default_args = Scenario()

print('Running simulation ...', end = ' => ')
results = single_run(default_args, warm_up=250)
print('simulation complete.')

results

Running simulation ... => 

AttributeError: 'Scenario' object has no attribute 'now'